# Jonathan Halverson
# Wednesday, December 6, 2017
# RNN applied to MNIST

In [4]:
import tensorflow as tf

In [1]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

In [2]:
learning_rate = 0.001

In [5]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

In [6]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [7]:
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)